In [1]:
from pathlib import Path
from src.blocks import BlockAnalyzer, Block, block_decomposition, Decomposer
from src.positional_strings import positional_string_from_block, PositionalString

In [2]:
path_blocks = Path("out/blocks/GC00000011_8.json")
list_blocks = BlockAnalyzer()._load_list_blocks(path_blocks)

In [3]:
list_blocks

[Block(K=(2, 3), i=0, j=32, label='---------------------------------'),
 Block(K=(0, 1), i=0, j=172, label='ATGAGATTGCGCAAGCGTTGCTTAATAACACGTATACGTTGTTTAACGTGCCGTAGGCCGGATAAGGCGTTCACGCCGCATCCGGCAGTTGGCGCACAATGCCTGATGCGACGCTTAACGCGTCTTATCATGCCTACAGGTTTGTGCCGAACCGTAGGCCGGATAAGGCGTTC'),
 Block(K=(0, 1, 2), i=33, j=34, label='AT'),
 Block(K=(0, 1, 2), i=36, j=70, label='CGTTGTTTAACGTGCCGTAGGCCGGATAAGGCGTT'),
 Block(K=(0, 1, 2), i=72, j=115, label='ACGCCGCATCCGGCAGTTGGCGCACAATGCCTGATGCGACGCTT'),
 Block(K=(2, 3), i=105, j=173, label='ATGCGACGCTTGACGCGTCTTATCATGCCTACAAGTCTGTGCCGAACCGTAGGCCGGATAAGGCGTTCA'),
 Block(K=(0, 1, 2, 3), i=105, j=115, label='ATGCGACGCTT'),
 Block(K=(0, 1, 2, 3), i=117, j=137, label='ACGCGTCTTATCATGCCTACA'),
 Block(K=(0, 1, 2, 3), i=139, j=140, label='GT'),
 Block(K=(0, 2), i=142, j=203, label='TGTGCCGAACCGTAGGCCGGATAAGGCGTTCACGCCGCATCCGGCAGTTGGCGCACAATGCC'),
 Block(K=(0, 2, 3), i=142, j=173, label='TGTGCCGAACCGTAGGCCGGATAAGGCGTTCA'),
 Block(K=(0, 1, 2, 3), i=142, j=17

In [4]:
decomposer=Decomposer()
new_blocks = decomposer(list_blocks)

In [5]:
len(new_blocks), len(list_blocks)

(45, 16)

In [6]:
import dataclasses
dataclasses.astuple(new_blocks[0])

((2, 3), 141, 173, 'CTGTGCCGAACCGTAGGCCGGATAAGGCGTTCA')

In [7]:
from collections import defaultdict
d = defaultdict(int)
for block in new_blocks:
    d[ block ] +=1
d

defaultdict(int,
            {Block(K=(2, 3), i=141, j=173, label='CTGTGCCGAACCGTAGGCCGGATAAGGCGTTCA'): 1,
             Block(K=(0, 1, 2, 3), i=175, j=203, label='GCCGCATCCGGCAGTTGGCGCACAATGCC'): 1,
             Block(K=(0, 1), i=174, j=215, label='CGCCGCATCCGGCAGTTGGCGCACAATGCC------------'): 1,
             Block(K=(0, 1), i=0, j=35, label='ATGAGATTGCGCAAGCGTTGCTTAATAACACGTATA'): 1,
             Block(K=(0, 1, 2, 3), i=142, j=172, label='TGTGCCGAACCGTAGGCCGGATAAGGCGTTC'): 1,
             Block(K=(0, 1), i=71, j=172, label='CACGCCGCATCCGGCAGTTGGCGCACAATGCCTGATGCGACGCTTAACGCGTCTTATCATGCCTACAGGTTTGTGCCGAACCGTAGGCCGGATAAGGCGTTC'): 1,
             Block(K=(0, 1), i=0, j=32, label='ATGAGATTGCGCAAGCGTTGCTTAATAACACGT'): 1,
             Block(K=(2, 3), i=105, j=173, label='ATGCGACGCTTGACGCGTCTTATCATGCCTACAAGTCTGTGCCGAACCGTAGGCCGGATAAGGCGTTCA'): 1,
             Block(K=(0, 1), i=116, j=172, label='AACGCGTCTTATCATGCCTACAGGTTTGTGCCGAACCGTAGGCCGGATAAGGCGTTC'): 1,
             Block(K=(0, 1, 2), i

___
## Block decomposition
Given a list of blocks, decompose intersections 